In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.metrics import classification_report, confusion_matrix

def load_and_prepare_data(train_path):
    df = pd.read_csv(train_path)
    df = df.rename(columns={"Label": "label"})
    return Dataset.from_pandas(df)

def load_and_prepare_test_data(test_path):
    df = pd.read_csv(test_path)
    df = df.rename(columns={"Label": "label"})
    return Dataset.from_pandas(df), df

def tokenize_dataset(dataset, tokenizer):
    def tokenize_function(examples):
        return tokenizer(examples['Sentence'], padding='max_length', truncation=True, max_length=128)
    tokenized = dataset.map(tokenize_function, batched=True)
    tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
    return tokenized

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), axis=1).numpy()
    report = classification_report(labels, preds, output_dict=True)
    acc = report['accuracy']
    f1 = report['macro avg']['f1-score']
    return {'accuracy': acc, 'f1_macro': f1}

def train_and_evaluate(model_name, train_dataset, test_datasets, raw_test_dfs, output_base_dir):
    print(f"\n--- Fine-tuning model: {model_name} ---")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

    tokenized_train = tokenize_dataset(train_dataset, tokenizer)

    training_args = TrainingArguments(
        output_dir=f"{output_base_dir}/model",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        weight_decay=0.01,
        load_best_model_at_end=False,
        logging_dir=f"{output_base_dir}/logs",
        logging_steps=50,
        save_total_limit=2,
        seed=42,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        compute_metrics=compute_metrics,
    )

    # Treniraj model
    trainer.train()

    # Spremi model nakon treninga
    trainer.save_model()

    # Evaluiraj i predvidi na svakom test skupu
    for i, (test_dataset, raw_test_df) in enumerate(zip(test_datasets, raw_test_dfs), start=1):
        print(f"\nEvaluacija na test skupu test-{i}")

        tokenized_test = tokenize_dataset(test_dataset, tokenizer)
        eval_results = trainer.evaluate(eval_dataset=tokenized_test)
        print(f"Evaluacija: {eval_results}")

        predictions_output = trainer.predict(tokenized_test)
        preds = torch.argmax(torch.tensor(predictions_output.predictions), axis=1).numpy()
        labels = predictions_output.label_ids

        print("Confusion Matrix:")
        print(confusion_matrix(labels, preds))

        print("\nClassification Report:")
        print(classification_report(labels, preds, target_names=['negative', 'neutral', 'positive']))

        # Spremi predikcije u CSV
        output_df = raw_test_df.copy()
        output_df['predicted_label'] = preds
        output_df['correct'] = output_df['label'] == output_df['predicted_label']
        output_csv = f"{output_base_dir}/predictions_test_{i}.csv"
        output_df.to_csv(output_csv, index=False)
        print(f"Predikcije spremljene u {output_csv}")

if __name__ == "__main__":
    # Učitaj trening skupove zasebno
    train_files = {
        "train_combined": "TRAIN.csv",
        "train_2": "train-2.csv"
    }

    # Učitaj test skupove
    test_files = ["test-1.csv", "test-2.csv", "test-3.csv"]
    test_datasets = []
    raw_test_dfs = []
    for f in test_files:
        ds, df = load_and_prepare_test_data(f)
        test_datasets.append(ds)
        raw_test_dfs.append(df)

    model_name = "classla/bcms-bertic"

    # Za svaki trening skup treniraj i evaluiraj model na sva tri testa
    for train_name, train_path in train_files.items():
        print(f"\n\n=== Treniranje i evaluacija za trening skup: {train_name} ===")
        train_dataset = load_and_prepare_data(train_path)
        output_dir = f"results_{train_name}"
        train_and_evaluate(model_name, train_dataset, test_datasets, raw_test_dfs, output_dir)




=== Treniranje i evaluacija za trening skup: train_combined ===

--- Fine-tuning model: classla/bcms-bertic ---


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7577 [00:00<?, ? examples/s]

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
50,0.929600
100,0.843100
150,0.744100
200,0.645000
250,0.633000
300,0.641400
350,0.618200
400,0.594200
450,0.578800
500,0.484900


...

Map:   0%|          | 0/793 [00:00<?, ? examples/s]

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Evaluacija: {'eval_loss': 0.9036539793014526, 'eval_accuracy': 0.7112232030264817, 'eval_f1_macro': 0.7055643128874013, 'eval_runtime': 18.7008, 'eval_samples_per_second': 42.405, 'eval_steps_per_second': 1.337, 'epoch': 3.0}


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confusion Matrix:
[[204  53  10]
 [ 17 239   7]
 [ 13 129 121]]

Classification Report:
              precision    recall  f1-score   support

    negative       0.87      0.76      0.81       267
     neutral       0.57      0.91      0.70       263
    positive       0.88      0.46      0.60       263

    accuracy                           0.71       793
   macro avg       0.77      0.71      0.71       793
weighted avg       0.77      0.71      0.71       793

Predikcije spremljene u results_train_2/predictions_test_3.csv
